# **5. Modelo Original**

In [3]:
import numpy as np                  # Para manipular arreglos
import pandas as pd                # Para trabajar con DataFrames
from sklearn.ensemble import GradientBoostingRegressor  # Meta-modelo: regresión cuantil
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score  # Métricas
from sklearn.model_selection import train_test_split    # División de datos
import joblib                         # Para cargar modelos guardados


In [2]:
# Cargar datos
df = pd.read_csv(r'C:\Users\TAWTOCA\OneDrive - Universidad del Norte\Documentos\Visual Folders\Visual Files\Rentabilidad_filtrado.csv')

# Definir variables predictoras y respuesta
# Definir variables predictoras y respuesta

X = df[["APORTES_RECIBIDOS", "PRECIERRE_FONDO_DIA_T", "TIPO_PARTICIPACION", "NUMERO_INVERSIONISTAS", "TIPO_ENTIDAD"]]  # variables predictoras
y = df["RETIROS_REDENCIONES"]

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42)



Se cargan desde disco los modelos previamente entrenados: los modelos XGBoost y Random Forest. 

> Los modelos fueron previamiente entrenados en la sección *3. Modelos Benchmarks*

In [5]:

ruta_modelos = 'modelos_guardados'
xgboost = joblib.load(f'{ruta_modelos}/xgboost.pkl')
random = joblib.load(f'{ruta_modelos}/random.pkl')


Se generan las predicciones de los modelos base (xgboost y random) sobre el conjunto de entrenamiento. Estas predicciones no son las predicciones finales, sino que serán utilizadas como nuevas variables (o meta-características) para entrenar un modelo adicional. Por ello, las predicciones se combinan columna por columna usando `np.column_stack`, formando una nueva matriz de entrada llamada `X_meta_train`, que será la entrada para el meta-modelo.

In [6]:
# Obtener predicciones de los modelos base sobre el conjunto de entrenamiento
pred_train_1 = xgboost.predict(X_train)
pred_train_2 = random.predict(X_train)

# Apilar las predicciones para usarlas como entrada al meta-modelo
X_meta_train = np.column_stack((pred_train_1, pred_train_2))


Se define y entrena el meta-modelo, que en este caso es un `GradientBoostingRegressor` con función de pérdida tipo `quantile` (cuantil) y con 𝛼 = 0.5, lo que equivale a realizar una regresión a la mediana. Este tipo de modelo es útil para mitigar la influencia de valores atípicos y distribuciones asimétricas. El modelo se entrena utilizando como entrada las predicciones apiladas de los modelos base (`X_meta_train`) y como salida la variable respuesta original `y_train`.

In [7]:


# Meta-modelo: regresión cuantil (usando GradientBoosting con loss='quantile')
meta_model = GradientBoostingRegressor(loss='quantile', alpha=0.5, random_state=42)

# Entrenar meta-modelo con las predicciones de los modelos base
meta_model.fit(X_meta_train, y_train)


GradientBoostingRegressor(alpha=0.5, loss='quantile', random_state=42)

Luego del entrenamiento, se evalúa el modelo ensamblado sobre el conjunto de prueba. Para esto, primero se obtienen las predicciones de los modelos base sobre `X_test`, que posteriormente se apilan en `X_meta_test`. Esta nueva matriz de características es usada por el meta-modelo para generar las predicciones finales `y_pred`. De esta manera, el modelo ensamblado utiliza la combinación de dos modelos base para mejorar su capacidad predictiva.

In [8]:
# Obtener predicciones de los modelos base sobre el conjunto de prueba
pred_test_1 = xgboost.predict(X_test)
pred_test_2 = random.predict(X_test)

# Apilar las predicciones
X_meta_test = np.column_stack((pred_test_1, pred_test_2))

# Predicciones finales del modelo ensamblado
y_pred = meta_model.predict(X_meta_test)


En esta etapa se evalúa el rendimiento del modelo ensamblado sobre los conjuntos de entrenamiento y prueba. Se generan las predicciones del meta-modelo sobre `X_meta_train` y `X_meta_test`, y luego se calculan tres métricas: RMSE (raíz del error cuadrático medio), MAE (error absoluto medio) y R² (coeficiente de determinación). Estas métricas permiten comparar el ajuste del modelo sobre los datos que vio en entrenamiento y su capacidad de generalización a nuevos datos. 

In [11]:
# Predicciones sobre entrenamiento y prueba
y_train_pred = meta_model.predict(X_meta_train)

# Métricas de entrenamiento
rmse_train = np.sqrt(mean_squared_error(y_train, y_train_pred))
mae_train = mean_absolute_error(y_train, y_train_pred)
r2_train = r2_score(y_train, y_train_pred)

# Métricas de prueba
rmse_test = np.sqrt(mean_squared_error(y_test, y_pred))
mae_test = mean_absolute_error(y_test, y_pred)
r2_test = r2_score(y_test, y_pred)

# Imprimir resultados
print("🔹 Entrenamiento:")
print(f"  RMSE: {rmse_train:.2f}")
print(f"  MAE:  {mae_train:.2f}")
print(f"  R²:   {r2_train:.2f}\n")

print("🔹 Prueba:")
print(f"  RMSE: {rmse_test:.2f}")
print(f"  MAE:  {mae_test:.2f}")
print(f"  R²:   {r2_test:.2f}")


🔹 Entrenamiento:
  RMSE: 20755100723.41
  MAE:  3945795255.24
  R²:   0.79

🔹 Prueba:
  RMSE: 21044482747.49
  MAE:  5273043709.55
  R²:   0.77


### Interpretación de Resultados

Los resultados obtenidos para el modelo ensamblado (basado en `XGBoost`, `Random Forest` y un meta-modelo `GradientBoostingRegressor`) muestran un desempeño sólido tanto en el conjunto de entrenamiento como en el de prueba:

#### Entrenamiento

* **RMSE: 20.76 mil millones**
  El error cuadrático medio indica que, en promedio, las predicciones del modelo difieren del valor real en aproximadamente 20.76 mil millones de unidades monetarias. Aunque parece elevado, esto puede estar influenciado por la escala de los datos (por ejemplo, si los valores de los retiros/redenciones son grandes).

* **MAE: 3.95 mil millones**
  El error absoluto medio indica que, en promedio, el modelo se equivoca por unos 3.95 mil millones en cada predicción. Este valor es más interpretable que el RMSE ya que no penaliza tanto los errores extremos.

* **R²: 0.79**
  El modelo es capaz de explicar el **79% de la variabilidad** en la variable respuesta (`RETIROS_REDENCIONES`). Esto indica un buen ajuste a los datos de entrenamiento.

####  Prueba

* **RMSE: 21.04 mil millones**
* **MAE: 5.27 mil millones**
* **R²: 0.77**

En el conjunto de prueba, las métricas son ligeramente peores, como es esperable en modelos bien ajustados. El R² baja ligeramente a 0.77, lo que sigue siendo un **indicador de buen poder predictivo**. La diferencia entre el MAE de entrenamiento y prueba sugiere que el modelo tiene un leve sobreajuste, pero no grave. La consistencia entre los conjuntos indica que el modelo generaliza bien.

#### Conclusión

El modelo ensamblado tiene un desempeño robusto y estable. Explica un alto porcentaje de la variabilidad de los retiros/redenciones y mantiene errores razonables en ambos conjuntos. Esto lo convierte en una opción válida para su aplicación práctica en el contexto de análisis de comportamiento financiero en FIC/FCP.

